# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298810075320                   -0.85    4.0
  2   -8.300253367480       -2.84       -1.25    1.0
  3   -8.300449486893       -3.71       -1.89    4.1
  4   -8.300462874615       -4.87       -2.77    1.8
  5   -8.300464315343       -5.84       -3.13    4.1
  6   -8.300464463413       -6.83       -3.29    4.8
  7   -8.300464547848       -7.07       -3.43    2.4
  8   -8.300464599743       -7.28       -3.58    1.0
  9   -8.300464629248       -7.53       -3.74    1.0
 10   -8.300464638342       -8.04       -3.89    1.9
 11   -8.300464643691       -8.27       -4.24    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64201526429                   -0.70    5.5
  2   -16.67867808208       -1.44       -1.14    1.8
  3   -16.67922918134       -3.26       -1.87    3.2
  4   -16.67928180960       -4.28       -2.77    4.1
  5   -16.67928613225       -5.36       -3.14    5.2
  6   -16.67928621406       -7.09       -3.53    1.4
  7   -16.67928622228       -8.08       -4.04    1.8


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32535885804                   -0.56    7.0
  2   -33.33274901054       -2.13       -0.99    1.0
  3   -33.33591397932       -2.50       -1.69    3.5
  4   -33.33689653922       -3.01       -2.44    5.8
  5   -33.33694200221       -4.34       -3.03    6.1
  6   -33.33694387612       -5.73       -3.85    7.4
  7   -33.33694392193       -7.34       -4.29    7.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298308285040                   -0.85    4.1
  2   -8.300256185729       -2.71       -1.59    1.5
  3   -8.300458098473       -3.69       -2.62    7.1
  4   -8.300417694327   +   -4.39       -2.42   11.8
  5   -8.300464478242       -4.33       -3.64    2.5
  6   -8.300464615680       -6.86       -4.08    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32607742714                   -0.56    6.4
  2   -33.30065115327   +   -1.59       -1.26    1.0
  3   -1.052692737564   +    1.51       -0.31    8.6
  4   -33.33462775048        1.51       -1.97    7.4
  5   -33.07127625258   +   -0.58       -1.28    5.4
  6   -32.69919116866   +   -0.43       -1.16    6.2
  7   -33.30803759619       -0.22       -1.77    7.2
  8   -33.33584764392       -1.56       -2.18    4.2
  9   -33.33456561396   +   -2.89       -2.15    3.6
 10   -33.33692631128       -2.63       -2.95    3.0
 11   -33.33693360421       -5.14       -3.07    2.6
 12   -33.33693824076       -5.33       -3.18    1.4
 13   -33.33694246866       -5.37       -3.60    3.0
 14   -33.33694360469       -5.94       -3.86    2.5
 15   -33.33694349795   +   -6.97       -4.20    2.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.